Work on clustering transcripts using their conservation score 1 kb upstream of TSS:
- slice
- intersect w/conservation scores
- 10 bp bins
- avg score w/in bins
- table
- standardize
- cluster! 

In [1]:
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed
import pandas as pd
import seaborn as sb
import statsmodels
import statsmodels.api as sm
%matplotlib inline

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
db = gffutils.FeatureDB('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.gtf.db')

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/gffutils/interface.py:161: UserWarning: It appears that this database has not had the ANALYZE sqlite3 command run on it. Doing so can dramatically speed up queries, and is done by default for databases created with gffutils >0.8.7.1 (this database was created with version 0.8.7.1) Consider calling the analyze() method of this object.
  "method of this object." % self.version)


In [3]:
transcripts = pybedtools_integration.to_bedtool(db.features_of_type('transcript')).saveas()

In [4]:
slopped = transcripts.slop(b=1000, s=True, genome='dm6').saveas()
#slopped might be good, but might need to do some subtraction??

In [5]:
genes = pybedtools_integration.to_bedtool(db.features_of_type('gene')).sort().merge().saveas()

In [6]:
diff = slopped.subtract(genes, s=False).saveas()

In [7]:
x = pybedtools_integration.tsses(db, merge_overlapping=True)

In [8]:
slop = x.slop(l=10000, r=0, s=True, genome='dm6').saveas('../../output/another_slop.bed')

In [9]:
#wig2bed < dm6.27way.phastCons.wigFix > dm6_phastcons.bed
phastcons = pybedtools.BedTool('../../output/dm6_phastcons.bed').saveas()

In [10]:
#intersections I need: 
#TSS w/exons, TSS w/introns, TSS w/intergenic

In [11]:
TSS_exons = x.intersect('../../output/dm6_exons.bed').saveas().to_dataframe()

In [12]:
#this doesn't work
TSS_introns = x.intersect('../../output/dm6_introns.bed').saveas().to_dataframe()

EmptyDataError: No columns to parse from file

In [13]:
TSS_intergenic = x.intersect('../../output/intergenic.bed').saveas().to_dataframe()

In [14]:
#intersect w/conservation scores:
#If I try to do here will fail, probably because I don't have enough memory? so doing it on local machine
#intersect = tssslop.intersect(phastcons, wb=True).saveas()

In [15]:
#import intersect file: (really big, splitting by chrom)
intersect_2L = pd.read_table('../../output/2L_intersect.bed', header=None)

In [16]:
intersect_2L.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,chr2L,0,7529,FBgn0031208,.,+,chr2L,443,444,id-1,0.152,1
1,chr2L,0,7529,FBgn0031208,.,+,chr2L,444,445,id-2,0.146,1
2,chr2L,0,7529,FBgn0031208,.,+,chr2L,445,446,id-3,0.139,1
3,chr2L,0,7529,FBgn0031208,.,+,chr2L,446,447,id-4,0.131,1
4,chr2L,0,7529,FBgn0031208,.,+,chr2L,447,448,id-5,0.123,1


In [17]:
#10bp bins(i.e. TSS-1…-10 = bin1, TSS-11…-20 = bin2 …)

clusters = pybedtools.BedTool.from_dataframe(intersect_2L).sort().cluster()

In [18]:
#Ugh this needs 50g of memory or something like that or else it doesn't work
clusters.head()

chr2L	0	7529	FBgn0031208	.	+	chr2L	4700	4701	id-2633	0.002	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4685	4686	id-2618	0.001	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4686	4687	id-2619	0.001	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4687	4688	id-2620	0.002	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4688	4689	id-2621	0.003	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4689	4690	id-2622	0.003	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4690	4691	id-2623	0.003	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4691	4692	id-2624	0.002	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4692	4693	id-2625	0.002	1	1
 chr2L	0	7529	FBgn0031208	.	+	chr2L	4693	4694	id-2626	0.001	1	1
 

In [19]:
clusterdf = clusters.to_dataframe()

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype bed are:
['chrom', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
but file has 13 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


In [20]:
clusterdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr2L,0,7529,FBgn0031208,.,+,chr2L,4700,4701,id-2633,0.002,1,1
1,chr2L,0,7529,FBgn0031208,.,+,chr2L,4685,4686,id-2618,0.001,1,1
2,chr2L,0,7529,FBgn0031208,.,+,chr2L,4686,4687,id-2619,0.001,1,1
3,chr2L,0,7529,FBgn0031208,.,+,chr2L,4687,4688,id-2620,0.002,1,1
4,chr2L,0,7529,FBgn0031208,.,+,chr2L,4688,4689,id-2621,0.003,1,1


In [28]:
clusterdf.groupby([12]).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr2L,0,7529,FBgn0031208,.,+,chr2L,4700,4701,id-2633,0.002,1,1
1,chr2L,0,7529,FBgn0031208,.,+,chr2L,4685,4686,id-2618,0.001,1,1
2,chr2L,0,7529,FBgn0031208,.,+,chr2L,4686,4687,id-2619,0.001,1,1
3,chr2L,0,7529,FBgn0031208,.,+,chr2L,4687,4688,id-2620,0.002,1,1
4,chr2L,0,7529,FBgn0031208,.,+,chr2L,4688,4689,id-2621,0.003,1,1
3939,chr2L,11951,21952,FBgn0263584,.,+,chr2L,15289,15290,id-11066,1.000,1,2
3940,chr2L,11951,21952,FBgn0263584,.,+,chr2L,15288,15289,id-11065,0.993,1,2
3941,chr2L,11951,21952,FBgn0263584,.,+,chr2L,15287,15288,id-11064,1.000,1,2
3942,chr2L,11951,21952,FBgn0263584,.,+,chr2L,15286,15287,id-11063,1.000,1,2
3943,chr2L,11951,21952,FBgn0263584,.,+,chr2L,15285,15286,id-11062,0.998,1,2


Questions for Justin: 
   - should I do a merge and collapse conservation scores? Or I could put them all in a list
   - what's going on with TSSes and introns? 